<a href="https://colab.research.google.com/github/marisbotero/sapiencia/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPROYECTO DISTRIAS

### LIBRERÍAS

In [1]:
import pandas as pd
import seaborn as sns

### LECTURA DEL DATASET

In [3]:
df = pd.read_excel('datos_distrias.xlsx')
df.head(2)

,NOMBRE_DOCUMENTO_MAESTRO,NOMBRE_CONSECUTIVO,TIPO_DOCUMENTO,NUMERO_DOCUMENTO,FECHA_DOCUMENTO,FECHA_VENCIMIENTO,IDENTIFICACION_TERCERO,NOMBRE_TERCERO,ID_CENTRO_COSTO,NOMBRE_CENTRO_COSTO,NOMBRE_CLASIFICACION,ID_PRODUCTO_SERVICIO,PRODUCTO_SERVICIO,CANTIDAD,VALOR_UNITARIO,VALOR_TOTAL,PORCENTAJE_DESCUENTO,VALOR_DESCUENTO,SUBTOTAL,IVA,TOTAL_NETO,UTILIDAD,RENTABILIDAD,COSTO,Columna2,CEDULA_VENDEDOR,NOMBRE_VENDEDOR,FORMA_DE_PAGO,TIPO,MARCA,LINEA,GRUPO,CLASE,NOMBRE_ESTABLECIMIENTO,ID_USUARIO_CREA,USUARIO_CREA,ID_TERCERO,ID_VENDEDOR,Columna1
0,FACTURA,FACTURA ELECTRONICA DEV,DEV,3077,2020-01-02,2020-02-01,900247351,ECHEVERRI INVERSIONES CAMEL SOCIEDAD POR ACCIO...,8003,SV VENTAS,GRAVADAS 19,383,SOLDADURA LIQ PVC 1/32,1.0,14321.0,14321.0,40.0,5728.4,8592.6,1633.0,10225.6,3179.6,37.0039,5413.0,3179.6,6869313.0,ACOSTA SUAREZ EDUARDO,CREDITO,NaN,017-GERFOR,NaN,1023-SOLDADURA PVC CEMENTO SOLVENTE,NaN,NaN,25785971,LEDA YIMARA ARGUMEDO DIAZ,4509,1856,NaN
1,FACTURA,FACTURA ELECTRONICA DEV,DEV,3077,2020-01-02,2020-02-01,900247351,ECHEVERRI INVERSIONES CAMEL SOCIEDAD POR ACCIO...,8003,SV VENTAS,GRAVADAS 19,164,ADAPTADOR PRES MACHO 1,4.0,1302.0,5208.0,40.0,2083.2,3124.8,594.0,3718.8,1212.8,38.8120,1912.0,1212.8,6869313.0,ACOSTA SUAREZ EDUARDO,CREDITO,NaN,017-GERFOR,NaN,1007-ACCESORIOS PRESION LISO-ROSCADO,NaN,NaN,25785971,LEDA YIMARA ARGUMEDO DIAZ,4509,1856,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222272 entries, 0 to 222271
Data columns (total 39 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NOMBRE_DOCUMENTO_MAESTRO  222272 non-null  object        
 1   NOMBRE_CONSECUTIVO        222272 non-null  object        
 2   TIPO_DOCUMENTO            222272 non-null  object        
 3   NUMERO_DOCUMENTO          222272 non-null  int64         
 4   FECHA_DOCUMENTO           222272 non-null  datetime64[ns]
 5   FECHA_VENCIMIENTO         222272 non-null  datetime64[ns]
 6   IDENTIFICACION_TERCERO    222272 non-null  object        
 7   NOMBRE_TERCERO            222272 non-null  object        
 8   ID_CENTRO_COSTO           222272 non-null  int64         
 9   NOMBRE_CENTRO_COSTO       222272 non-null  object        
 10  NOMBRE_CLASIFICACION      222272 non-null  object        
 11  ID_PRODUCTO_SERVICIO      222272 non-null  int64         
 12  PR

### LIMPIEZA E IMPUTACIÓN DE DATOS

In [5]:
#Transformar tipo de variable
df['NUMERO_DOCUMENTO']=df['NUMERO_DOCUMENTO'].astype(str)
df['CEDULA_VENDEDOR']=df['CEDULA_VENDEDOR'].astype(str)
df['ID_CENTRO_COSTO']=df['ID_CENTRO_COSTO'].astype(str)
df['ID_PRODUCTO_SERVICIO']=df['ID_PRODUCTO_SERVICIO'].astype(str)
df['ID_USUARIO_CREA']=df['ID_USUARIO_CREA'].astype(str)
df['ID_TERCERO']=df['ID_TERCERO'].astype(str)
df['ID_VENDEDOR']=df['ID_VENDEDOR'].astype(str)

In [6]:
#Valores nulos
null_values=df.isnull().sum()/len(df) #Valores nulos
null_values[null_values>0]

UTILIDAD                  0.000018
RENTABILIDAD              0.000018
COSTO                     0.000018
Columna2                  0.000018
NOMBRE_VENDEDOR           0.000868
TIPO                      0.845739
MARCA                     0.095415
LINEA                     0.359933
GRUPO                     0.027768
CLASE                     0.321853
NOMBRE_ESTABLECIMIENTO    0.795251
Columna1                  1.000000
dtype: float64

In [7]:
#Imputación de columnas
df = df.drop(['NOMBRE_ESTABLECIMIENTO','TIPO'], axis=1)

In [8]:
#Hay filas duplicadas?
l=[]
mask=df.duplicated()
for i in mask:
    if i==True:
        l.append(i)
print(len(l))

97


In [9]:
#Eliminar las filas dupliucadas
df = df.drop_duplicates()

In [10]:
#En la columna 'NOMBRE_DOCUMENTO_MAESTRO' se van a eliminar las filas correspondientes a notas crédito, pues es una devolución que no implica utilidades para la empresa
df = df.set_index('NOMBRE_DOCUMENTO_MAESTRO')
df = df.drop('NOTA CREDITO')
df.reset_index(level=0, inplace=True)

### EDA

In [11]:
df.describe()

,CANTIDAD,VALOR_UNITARIO,VALOR_TOTAL,PORCENTAJE_DESCUENTO,VALOR_DESCUENTO,SUBTOTAL,IVA,TOTAL_NETO,UTILIDAD,RENTABILIDAD,COSTO,Columna2,Columna1
count,212493.000000,2.124930e+05,2.124930e+05,212493.000000,2.124930e+05,2.124930e+05,2.124930e+05,2.124930e+05,2.124930e+05,2.124930e+05,2.124930e+05,2.124930e+05,0.0
mean,35.022012,4.450882e+04,4.729484e+05,8.779349,3.604622e+04,4.369021e+05,8.168787e+04,5.185900e+05,9.090986e+04,-5.265953e+01,3.459923e+05,9.090986e+04,NaN
std,311.370932,1.422137e+05,2.163091e+06,13.121995,3.601346e+05,1.998362e+06,3.789614e+05,2.376488e+06,3.980130e+05,2.339324e+04,1.659557e+06,3.980130e+05,NaN
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.108899e+07,-9.019149e+06,0.000000e+00,-1.108899e+07,NaN
25%,1.000000,4.500000e+03,1.965000e+04,0.000000,0.000000e+00,1.758820e+04,3.106500e+03,2.090000e+04,4.964845e+03,1.927170e+01,1.162000e+04,4.964845e+03,NaN
50%,5.000000,1.680000e+04,7.420168e+04,0.000000,0.000000e+00,6.675000e+04,1.211176e+04,7.921116e+04,1.679382e+04,2.615210e+01,4.747100e+04,1.679382e+04,NaN
75%,20.000000,3.605000e+04,2.863500e+05,12.000000,1.323600e+04,2.638700e+05,4.853800e+04,3.124009e+05,6.024000e+04,3.498690e+01,1.985370e+05,6.024000e+04,NaN
max,110000.000000,1.167893e+07,4.006800e+08,100.000000,7.596953e+07,4.006800e+08,7.612920e+07,4.768092e+08,4.871156e+07,1.000000e+02,3.519684e+08,4.871156e+07,NaN


### ANÁLISIS RFM

El análisis RFM permite determinar cuáles son los mejores clientes teniendo en cuenta cuándo realizaron su última compra, con qué frecuencia lo hacen y cuánto se gastan. El RFM permite entender mejor cómo funciona la base de clientes, conociendo su comportamiento.

Para aplicar la técnica RFM hay que tener en cuenta las tres principales variables: Recency, Frequency, Monetary.

 - Recencia: ¿Cuándo fue la última vez que el cliente hizo una compra?
 - Frecuencia: define el número de interacciones del cliente con la marca en un espacio determinado de tiempo.
 - Valor monetario: refleja la cantidad que se ha gastado el cliente en las compras hechas en ese espacio de tiempo.



#### RESENCIA

In [12]:
df['recency'] = pd.to_datetime('2021-11-13 08:26:00') - df['FECHA_VENCIMIENTO']

In [13]:
def recency(recency):
    res = str(recency).split(' ')[0]
    return(int(res))
df['recency'] = df['recency'].apply(recency)

In [14]:
df_recency = df[['NOMBRE_TERCERO','IDENTIFICACION_TERCERO','recency']]
df_recency = df_recency.drop_duplicates()

In [15]:
df_recency

,NOMBRE_TERCERO,IDENTIFICACION_TERCERO,recency
0,ECHEVERRI INVERSIONES CAMEL SOCIEDAD POR ACCIO...,900247351,651
3,GOMEZ MORA HELIODORO JOSE,890226,679
5,DAVID ORDOSGOITIA LEONARDO JALIL,78675754,680
8,COA GONZALEZ JACINTO,98613374,679
9,ASOCIACION COLEGIO MILITAR ALMIRANTE COLON,806007709,651
...,...,...,...
212474,CONSORCIO OBRAS VALLE INFERIOR DEL MAGDALENA,901420745,132
212483,CONSORCIO AQUA DE ORO,901332047,263
212484,CONSORCIO OBRAS VALLE INFERIOR DEL MAGDALENA,901420745,283
212489,CONSORCIO AQUA DE ORO,901332047,285


#### FRECUENCIA

In [16]:
freq = df.groupby(['NOMBRE_TERCERO','IDENTIFICACION_TERCERO'])['FECHA_VENCIMIENTO'].count()
df_freq = pd.DataFrame(freq).reset_index()

In [17]:
df_freq

,NOMBRE_TERCERO,IDENTIFICACION_TERCERO,FECHA_VENCIMIENTO
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,7
1,ALCALA HERNANDEZ PAOLA,1067842751,2
2,ALVAREZ FERRER LUIS,6890908,3
3,BARBOSA OSUNA MARINA,43692562,4
4,BELEÑO RAMBAO GLORIA,25872863,2
...,...,...,...
16588,ZUÑIGA HERRERA LADISLAO,78690273,2
16589,ZUÑIGA PEREZ ANGEL,6864474,7
16590,ZUÑIGA PEÑATE CESARDAVID,78705352,1
16591,ZUÑIGA RAMIRO,6874582,1


#### MONTO

In [18]:
monto = df.groupby(['NOMBRE_TERCERO','IDENTIFICACION_TERCERO'])['SUBTOTAL'].sum()
df_monto = pd.DataFrame(monto).reset_index()

In [19]:
df_monto

,NOMBRE_TERCERO,IDENTIFICACION_TERCERO,SUBTOTAL
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,4.956520e+05
1,ALCALA HERNANDEZ PAOLA,1067842751,5.556092e+06
2,ALVAREZ FERRER LUIS,6890908,5.436849e+06
3,BARBOSA OSUNA MARINA,43692562,3.782276e+06
4,BELEÑO RAMBAO GLORIA,25872863,1.681639e+06
...,...,...,...
16588,ZUÑIGA HERRERA LADISLAO,78690273,1.568450e+06
16589,ZUÑIGA PEREZ ANGEL,6864474,4.602804e+05
16590,ZUÑIGA PEÑATE CESARDAVID,78705352,1.728300e+03
16591,ZUÑIGA RAMIRO,6874582,1.090714e+06


#### DATASET RFM

In [20]:
finaldf = df_freq.merge(df_monto).merge(df_recency)

In [21]:
finaldf

,NOMBRE_TERCERO,IDENTIFICACION_TERCERO,FECHA_VENCIMIENTO,SUBTOTAL,recency
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,7,4.956520e+05,548
1,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,306
2,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,164
3,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,366
4,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,365
...,...,...,...,...,...
64179,ZUÑIGA PEREZ ANGEL,6864474,7,4.602804e+05,157
64180,ZUÑIGA PEREZ ANGEL,6864474,7,4.602804e+05,144
64181,ZUÑIGA PEÑATE CESARDAVID,78705352,1,1.728300e+03,331
64182,ZUÑIGA RAMIRO,6874582,1,1.090714e+06,621


In [22]:
finaldf.columns

Index(['NOMBRE_TERCERO', 'IDENTIFICACION_TERCERO', 'FECHA_VENCIMIENTO',
       'SUBTOTAL', 'recency'],
      dtype='object')

In [24]:
df_final = finaldf.rename(columns={'NOMBRE_TERCERO': 'Nombre_cliente', 'IDENTIFICACION_TERCERO': 'id_cliente','FECHA_VENCIMIENTO': 'Frecuencia',
       'SUBTOTAL': 'Monto', 'recency':'Recencia'})

In [25]:
df_final.head()

,Nombre_cliente,id_cliente,Frecuencia,Monto,Recencia
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,7,4.956520e+05,548
1,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,306
2,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,164
3,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,366
4,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,365


In [28]:
df_final['r_cuartil'] = pd.qcut(df_final['Recencia'], 4, ['1','2','3','4'])
df_final['f_cuartil'] = pd.qcut(df_final['Frecuencia'], 4, ['4','3','2','1'])
df_final['m_cuartil'] = pd.qcut(df_final['Monto'], 4, ['4','3','2','1'])

In [29]:
df_final.head()

,Nombre_cliente,id_cliente,Frecuencia,Monto,Recencia,r_cuartil,f_cuartil,m_cuartil
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,7,4.956520e+05,548,4,4,4
1,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,306,2,4,3
2,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,164,2,4,3
3,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,366,3,4,3
4,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,365,3,4,3


In [30]:
df_final['RFM_Score'] = df_final['r_cuartil'].astype(str)+ df_final['f_cuartil'].astype(str) + df_final['m_cuartil'].astype(str)
df_final.head()

,Nombre_cliente,id_cliente,Frecuencia,Monto,Recencia,r_cuartil,f_cuartil,m_cuartil,RFM_Score
0,\tEDIFICIO PORTO ALEGRE - PROPIEDAD HORIZONTAL,900429009,7,4.956520e+05,548,4,4,4,444
1,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,306,2,4,3,243
2,ALCALA HERNANDEZ PAOLA,1067842751,2,5.556092e+06,164,2,4,3,243
3,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,366,3,4,3,343
4,ALVAREZ FERRER LUIS,6890908,3,5.436849e+06,365,3,4,3,343


In [34]:
df_final.to_csv('df_final_rfm.csv', index=False)

In [33]:
# Filter out Top/Best cusotmers
df_final[df_final['RFM_Score']=='111'].sort_values('Monto', ascending=False).head()

,Nombre_cliente,id_cliente,Frecuencia,Monto,Recencia,r_cuartil,f_cuartil,m_cuartil,RFM_Score
12486,CONSORCIO AQUA DE ORO,901332047,359,1.338188e+09,149,1,1,1,111
12482,CONSORCIO AQUA DE ORO,901332047,359,1.338188e+09,131,1,1,1,111
12484,CONSORCIO AQUA DE ORO,901332047,359,1.338188e+09,136,1,1,1,111
12483,CONSORCIO AQUA DE ORO,901332047,359,1.338188e+09,154,1,1,1,111
12489,CONSORCIO AQUA DE ORO,901332047,359,1.338188e+09,135,1,1,1,111
